# Batch SQL Gold


In [0]:

USE IDENTIFIER(:catalog ||'.'|| :schema)


# Sales

In [0]:
MERGE INTO gold_sales AS T
USING (
  SELECT
    sha2(concat_ws('|',Sales_ID,Registration_TimeStamp),256) as Sales_SK,
    A.Sales_ID,
    B.Customer_SK,
    C.Product_SK,
    A.Revenue,
    A.Units_Sold,
    greatest(A.Moviment_Date,B.Moviment_Date,C.Moviment_Date) AS Moviment_Date,
    A.Registration_TimeStamp AS Sales_Timestamp
  FROM silver_sales AS A
  LEFT JOIN gold_customers AS B
    ON A.Customer_ID = B.Customer_ID AND A.Registration_TimeStamp BETWEEN B.StartDate AND B.EndDate
  LEFT JOIN gold_products AS C
    ON A.Product_ID = C.Product_ID AND A.Registration_TimeStamp BETWEEN C.StartDate AND C.EndDate
  WHERE A.Moviment_Date = :moviment_date OR B.Moviment_Date = :moviment_date OR C.Moviment_Date = :moviment_date
) AS S
ON T.Sales_SK = S.Sales_SK
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
9,0,0,9


In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project 
  AND Table_name = 'gold_sales'